In [75]:
import pandas as pd
import numpy as np
import pylab as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn import cluster
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn import pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from datetime import datetime
import time 
import xgboost as xgb

%matplotlib inline

# Calculate average speed, and use that 
#class AvgSpeed(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        if 'Trip_distance' not in X.columns:
            raise KeyError('X Dataframe needs to have column "Trip_distance"')
        self.avg = (y / X['Trip_distance']).mean()
        return self
    def predict(self, X):
        if 'Trip_distance' not in X.columns:
            raise KeyError('X Dataframe needs to have column "Trip_distance"')
        return self.avg * X['Trip_distance']

# Load data

In [126]:
# load training
training = pd.read_csv('C:/Users/siyam/Downloads/UmojaHackYassir/UmojaHack25July/Train.csv', sep = ';')
training['Timestamp'] = pd.to_datetime(training['Timestamp'])
training.head()

,ID,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA,date
0,000FLWA8,2019-12-04 22:01:00,3.258,36.777,3.003,36.718,39627,2784,2019-12-04
1,000RGOAM,2019-12-11 00:37:00,3.087,36.707,3.081,36.727,3918,576,2019-12-11
2,001QSGIH,2019-11-23 22:36:00,3.144,36.739,3.088,36.742,7265,526,2019-11-23
3,002ACV6R,2019-12-01 07:43:00,3.239,36.784,3.054,36.763,23350,3130,2019-12-01
4,0039Y7A8,2019-12-17 22:30:00,2.912,36.707,3.207,36.698,36613,2138,2019-12-17


In [127]:
training1 = pd.read_csv('C:/Users/siyam/Downloads/UmojaHackYassir/UmojaHack25July/Train-IM2.csv', sep = ';')
training1['Timestamp'] = pd.to_datetime(training1['Timestamp'])
training1.head()

,ID,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA,date,Time
0,000FLWA8,2019-12-04 22:01:00,3.258,36.777,3.003,36.718,39627,2784,2019/12/04,79260
1,000RGOAM,2019-12-11 00:37:00,3.087,36.707,3.081,36.727,3918,576,2019/12/11,2220
2,001QSGIH,2019-11-23 22:36:00,3.144,36.739,3.088,36.742,7265,526,2019/11/23,81360
3,002ACV6R,2019-12-01 07:43:00,3.239,36.784,3.054,36.763,23350,3130,2019/12/01,27780
4,0039Y7A8,2019-12-17 22:30:00,2.912,36.707,3.207,36.698,36613,2138,2019/12/17,81000


# pre-processing

In [128]:
training1.isnull().sum()

ID                 0
Timestamp          0
Origin_lat         0
Origin_lon         0
Destination_lat    0
Destination_lon    0
Trip_distance      0
ETA                0
date               0
Time               0
dtype: int64

In [129]:
# sample set 
sample_set = pd.read_csv('C:/Users/siyam/Downloads/UmojaHackYassir/UmojaHack25July/SampleSubmission.csv').set_index('ID')
sample_set.head()

,ETA
ID,
000V4BQX,0
003WBC5J,0
004O4X3A,0
006CEI5B,0
009G0M2T,0


In [130]:
# testing data
testing = pd.read_csv('C:/Users/siyam/Downloads/UmojaHackYassir/UmojaHack25July/Test.csv', sep =',').set_index('ID')
testing['Timestamp'] = pd.to_datetime(testing['Timestamp'])
testing.head()

,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance
ID,,,,,,
000V4BQX,2019-12-21 05:52:37+00:00,2.981,36.688,2.978,36.754,17549
003WBC5J,2019-12-25 21:38:53+00:00,3.032,36.769,3.074,36.751,7532
004O4X3A,2019-12-29 21:30:29+00:00,3.035,36.711,3.010,36.758,10194
006CEI5B,2019-12-31 22:51:57+00:00,2.902,36.738,3.208,36.698,32768
009G0M2T,2019-12-28 21:47:22+00:00,2.860,36.692,2.828,36.696,4513


In [166]:
# testing data
testing1 = pd.read_csv('C:/Users/siyam/Downloads/UmojaHackYassir/UmojaHack25July/Test-IM.csv', sep =';')
testing1['Timestamp'] = pd.to_datetime(testing1['Timestamp'])
testing1.head()

,ID,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,date,Time
0,000V4BQX,2019-12-21 07:52:00,2.981,36.688,2.978,36.754,17549,2019/12/21,28320
1,003WBC5J,2019-12-25 23:38:00,3.032,36.769,3.074,36.751,7532,2019/12/25,85080
2,004O4X3A,2019-12-29 23:30:00,3.035,36.711,3.010,36.758,10194,2019/12/29,84600
3,006CEI5B,2020-01-01 00:51:00,2.902,36.738,3.208,36.698,32768,2020/01/01,3060
4,009G0M2T,2019-12-28 23:47:00,2.860,36.692,2.828,36.696,4513,2019/12/28,85620


In [132]:
testing1.isnull().sum()

Timestamp          0
Origin_lat         0
Origin_lon         0
Destination_lat    0
Destination_lon    0
Trip_distance      0
date               0
Time               0
dtype: int64

In [163]:
# weather data
weather = pd.read_csv('C:/Users/siyam/Downloads/UmojaHackYassir/UmojaHack25July/Weather.csv')
weather.head()

,date,dewpoint_2m_temperature,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m
0,2019-11-01,290.630524,296.434662,294.125061,101853.617188,292.503998,100806.351562,0.004297,3.561323,0.941695
1,2019-11-02,289.135284,298.432404,295.551666,101225.164062,293.337921,100187.250000,0.001767,5.318593,3.258237
2,2019-11-03,287.667694,296.612122,295.182831,100806.617188,293.674316,99771.414062,0.000797,8.447649,3.172982
3,2019-11-04,287.634644,297.173737,294.368134,101240.929688,292.376221,100200.843750,0.000393,5.991428,2.236700
4,2019-11-05,286.413788,294.284851,292.496979,101131.750000,289.143066,100088.500000,0.004658,6.962730,2.655364


# Make train and OOT set

In [135]:
# make sure sorted by date
#train = training.sort_values('Timestamp')
#train = training.drop(['date'], 1)
#oot_df = train.drop(["Timestamp"])
#oot_df = training.iloc[-25178:]
#oot_df.shape,train.shape

# Model fitting

In [174]:
training1.columns

Index(['ID', 'Timestamp', 'Origin_lat', 'Origin_lon', 'Destination_lat',
       'Destination_lon', 'Trip_distance', 'ETA', 'date', 'Time'],
      dtype='object')

In [175]:
# Get the data
from sklearn.model_selection import train_test_split
y = training1['ETA']
X = training1[['Origin_lat', 'Origin_lon', 'Destination_lat',
       'Destination_lon', 'Trip_distance','Time']]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)


In [176]:
y.shape

(83924,)

In [177]:
X_test.shape

(25178, 6)

In [178]:
test = testing1.drop(["Timestamp", "date"],1)

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Hypertuning model

In [179]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30,0.40] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15,20,30,40,50],
 "min_child_weight" : [ 1, 3, 5, 7 ,9,10,11,12],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4,1,1.5,2],
 "colsample_bytree" : [ 0.1,0.2,0.3, 0.4, 0.5 , 0.7,0.8,0.9]
 
    
}

In [180]:
## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb
import xgboost
boost = xgb.XGBRegressor()

In [181]:
random_search=RandomizedSearchCV(boost,param_distributions=params,n_iter=5,n_jobs=-1,cv=5,verbose=3)

In [182]:
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.0min finished


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n...
                                          scale_pos_weight=None, subsample=None,
                                          tree_method=None,
                                          validat

In [183]:
random_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0.0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.15, max_delta_step=0, max_depth=12,
             min_child_weight=10, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### LlNEAR REGRESSION

In [26]:
# Fit model
lm_reg = LinearRegression()
##Fitting the model with train dataset
lm_reg.fit(X_train, y_train)


LinearRegression()

In [27]:
# Getting predicions from the X_test
pred = lm_reg.predict(X_test)
#checking score
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred, squared=False)

244.97242574818063

In [28]:
# run test data through
submission = test.copy()
submission['ETA'] = lm_reg.predict(test)
submission.head()

,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA
ID,,,,,,
000V4BQX,2.981,36.688,2.978,36.754,17549,1331.461013
003WBC5J,3.032,36.769,3.074,36.751,7532,803.947890
004O4X3A,3.035,36.711,3.010,36.758,10194,937.792883
006CEI5B,2.902,36.738,3.208,36.698,32768,2112.036476
009G0M2T,2.860,36.692,2.828,36.696,4513,572.838355


In [29]:
# save
submission[['ETA']].to_csv('lm_submmision.csv')

### XGBOOST

In [143]:
import xgboost as xgb
from tqdm import tqdm
boost = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0.0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=20,
             min_child_weight=9, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
##Fitting the model with train dataset
boost = boost.fit(X_train, y_train)

## PREDICTIONS AND SCORE

In [144]:
# Getting predicions from the X_test
pred0 = boost.predict(X_test)
#checking score
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred0, squared=False)

137.3409561472998

## SUBMISSIONS

In [150]:
# run test data through
submission0 = test.copy()
submission0['ETA'] = boost.predict(test)
submission0.head()

,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,Time,ETA
ID,,,,,,,
000V4BQX,2.981,36.688,2.978,36.754,17549,28320,1490.426758
003WBC5J,3.032,36.769,3.074,36.751,7532,85080,857.479919
004O4X3A,3.035,36.711,3.010,36.758,10194,84600,1010.636414
006CEI5B,2.902,36.738,3.208,36.698,32768,3060,1881.665894
009G0M2T,2.860,36.692,2.828,36.696,4513,85620,693.871704


In [151]:
# save
submission0[['ETA']].to_csv('xgb_submmision.csv')

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Merged Training with the weather dataset

In [167]:
train_merge =  pd.merge(training, weather, on='date', how='inner')

In [168]:
train_merge.head()

,ID,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA,date,dewpoint_2m_temperature,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m
0,000FLWA8,2019-12-04 22:01:00,3.258,36.777,3.003,36.718,39627,2784,2019-12-04,283.047546,288.319153,286.984436,100523.0625,286.00177,99474.21875,0.00578,1.914582,2.437786
1,007N9ZND,2019-12-04 00:23:00,3.207,36.698,2.955,36.757,29007,2474,2019-12-04,283.047546,288.319153,286.984436,100523.0625,286.00177,99474.21875,0.00578,1.914582,2.437786
2,01NOTKQS,2019-12-04 22:48:00,2.906,36.710,3.049,36.750,19087,1728,2019-12-04,283.047546,288.319153,286.984436,100523.0625,286.00177,99474.21875,0.00578,1.914582,2.437786
3,02269DJO,2019-12-04 07:53:00,3.168,36.724,3.091,36.738,11330,929,2019-12-04,283.047546,288.319153,286.984436,100523.0625,286.00177,99474.21875,0.00578,1.914582,2.437786
4,029SZ5Y0,2019-12-04 00:40:00,3.003,36.718,2.982,36.695,4372,530,2019-12-04,283.047546,288.319153,286.984436,100523.0625,286.00177,99474.21875,0.00578,1.914582,2.437786


# Merge test with the weather data

In [169]:
testing1.head()

,ID,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,date,Time
0,000V4BQX,2019-12-21 07:52:00,2.981,36.688,2.978,36.754,17549,2019/12/21,28320
1,003WBC5J,2019-12-25 23:38:00,3.032,36.769,3.074,36.751,7532,2019/12/25,85080
2,004O4X3A,2019-12-29 23:30:00,3.035,36.711,3.010,36.758,10194,2019/12/29,84600
3,006CEI5B,2020-01-01 00:51:00,2.902,36.738,3.208,36.698,32768,2020/01/01,3060
4,009G0M2T,2019-12-28 23:47:00,2.860,36.692,2.828,36.696,4513,2019/12/28,85620


In [171]:
test_merge =  pd.merge(testing1, weather, on='date', how='outer')

In [172]:
test_merge.head()

,ID,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,date,Time,dewpoint_2m_temperature,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m
0,000V4BQX,2019-12-21 07:52:00,2.981,36.688,2.978,36.754,17549.0,2019/12/21,28320.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00BRKED8,2019-12-21 22:06:00,3.013,36.762,3.082,36.727,9878.0,2019/12/21,79560.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00HS3XXF,2019-12-21 00:24:00,3.099,36.744,3.099,36.742,4002.0,2019/12/21,1440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00WNC10D,2019-12-21 07:51:00,3.248,36.783,3.059,36.785,25461.0,2019/12/21,28260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01OZ7SN8,2019-12-21 22:41:00,3.059,36.756,3.102,36.726,10323.0,2019/12/21,81660.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
test_new = test_merge.drop(['Timestamp', 'date'], 1)

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Re-fitting model with the merge dataframe


In [ ]:
train_merge.columns

In [ ]:
# Get the data
from sklearn.model_selection import train_test_split
y = train_merge['ETA']
X = train_merge[[ 'Origin_lat', 'Origin_lon', 'Destination_lat',
       'Destination_lon', 'Trip_distance',
       'dewpoint_2m_temperature', 'maximum_2m_air_temperature',
       'mean_2m_air_temperature', 'mean_sea_level_pressure',
       'minimum_2m_air_temperature', 'surface_pressure', 'total_precipitation',
       'u_component_of_wind_10m', 'v_component_of_wind_10m', 'Time']]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)


//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30,0.40] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15,20,30,40,50],
 "min_child_weight" : [ 1, 3, 5, 7 ,9,10,11,12],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4,1,1.5,2],
 "colsample_bytree" : [ 0.1,0.2,0.3, 0.4, 0.5 , 0.7,0.8,0.9]
 
    
}

In [ ]:
## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb
import xgboost
boost = xgb.XGBRegressor()

In [ ]:
random_search=RandomizedSearchCV(boost,param_distributions=params,n_iter=5,n_jobs=-1,cv=5,verbose=3)

In [ ]:
random_search.fit(X_train,y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
import xgboost as xgb
from tqdm import tqdm
boost = xgb.XGBRegressor()
##Fitting the model with train dataset
boost = boost.fit(X_train, y_train)

In [ ]:
# Getting predicions from the X_test
pred0 = boost.predict(X_test)
#checking score
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred0, squared=False)

In [ ]:
# run test data through
submission2 = test_new.copy()
submission2['ETA'] = boost.predict(test_new)
submission2.head()

In [ ]:
# save
submission2[['ETA']].to_csv('merge_submmision.csv')

# LlNEAR REGRESSION

In [ ]:
# Fit model
lm_reg = LinearRegression()
##Fitting the model with train dataset
lm_reg.fit(X_train, y_train)


In [ ]:
# Getting predicions from the X_test
pred = lm_reg.predict(X_test)
#checking score
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred, squared=False)

In [ ]:
from sklearn.svm import SVC
LinearSVC = SVC(kernel="linear", C = 1)
##Fitting the model with train dataset
LinearSVC = LinearSVC.fit(X_train, y_train)

In [ ]:
# Getting predicions from the X_test
pred1 = LinearSVC.predict(X_test)
#checking score
mean_squared_error(y_test, pred1, squared=False)